In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import random

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import coo_matrix

from sklearn.base import BaseEstimator
from sklearn.model_selection import RandomizedSearchCV

In [3]:
os.chdir('/home/walml/repos/recommender_hack')

In [4]:
# !pip install lightfm

In [5]:
from lightfm.evaluation import auc_score, precision_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm import LightFM
from lightfm.data import Dataset

In [6]:
# df = pd.read_csv('normal.csv')
df = pd.read_csv('power.csv')

In [7]:
data = Dataset()
data.fit(df['comment_user_id'], df['comment_focus_id'])
data.model_dimensions()

(386, 43757)

In [8]:
# feature_df = pd.read_csv('notebooks/votes.csv')

In [9]:
# feature_cols = ['subject_id', 'bar_no', 'bar_strong', 'bar_weak',
#        'bulge-size_dominant', 'bulge-size_large', 'bulge-size_moderate',
#        'bulge-size_none', 'bulge-size_small', 'disk-edge-on_no',
#        'disk-edge-on_yes', 'edge-on-bulge_boxy', 'edge-on-bulge_none',
#        'edge-on-bulge_rounded', 'has-spiral-arms_no',
#        'has-spiral-arms_yes', 'how-rounded_cigar-shaped',
#        'how-rounded_in-between', 'how-rounded_round', 'merging_both-v1',
#        'merging_major-disturbance', 'merging_merger',
#        'merging_minor-disturbance', 'merging_neither-v1', 'merging_none',
#        'merging_tidal-debris-v1', 'smooth-or-featured_artifact',
#        'smooth-or-featured_featured-or-disk', 'smooth-or-featured_smooth',
#        'spiral-arm-count_1', 'spiral-arm-count_2', 'spiral-arm-count_3',
#        'spiral-arm-count_4', 'spiral-arm-count_cant-tell',
#        'spiral-arm-count_more-than-4', 'spiral-winding_loose',
#        'spiral-winding_medium', 'spiral-winding_tight']

In [10]:
# feature_df = feature_df[feature_cols]

In [11]:
t = df['comment_focus_id'].value_counts() > 1
t[t]

21660747.0    True
35622867.0    True
21666161.0    True
26440689.0    True
28479582.0    True
              ... 
27363364.0    True
20925112.0    True
32505491.0    True
20925162.0    True
21667356.0    True
Name: comment_focus_id, Length: 13969, dtype: bool

In [12]:
# feature_df['comment_focus_id'] = feature_df['subject_id'].astype(int)

In [13]:
# df_2 = pd.merge(df, feature_df, how='inner', on='comment_focus_id')

In [14]:
# df_2

In [113]:
def get_train_test(df, min_item_interactions=5):
    has_multiple_comments = df['comment_focus_id'].value_counts() > min_item_interactions
    multiple_comments = has_multiple_comments[has_multiple_comments]  # hilarious
    print(len(df))
    df = df[df['comment_focus_id'].isin(multiple_comments.index)]
    print(len(df))
    if 'weights' in df.columns.values:
        data_pairs = list(zip(df['comment_user_id'], df['comment_focus_id'], df['weights']))
    else:
        data_pairs = list(zip(df['comment_user_id'], df['comment_focus_id']))
#     random.shuffle(data_pairs)

    n_pairs = len(data_pairs)
    n_split = int(n_pairs * 0.8)
    train = data_pairs[:n_split]
    test = data_pairs[n_split:]
    return train, test, data_pairs

In [21]:
train, test, _ = get_train_test(df)

68449
5139


In [22]:
(train_interactions, _) = data.build_interactions(train)
print(repr(train_interactions))

(test_interactions, _) = data.build_interactions(test)
print(repr(test_interactions))


<386x43757 sparse matrix of type '<class 'numpy.int32'>'
	with 4111 stored elements in COOrdinate format>
<386x43757 sparse matrix of type '<class 'numpy.int32'>'
	with 1028 stored elements in COOrdinate format>


In [119]:
# https://scikit-learn.org/stable/developers/develop.html
class LightEstimator(BaseEstimator):
    
    def __init__(self, data, no_components=10, loss='warp', epochs=1):
        self.model = LightFM(loss=loss, no_components=no_components)
        self.data = data  # for ids only
        self.no_components = no_components
        self.loss=loss
        self.epochs = epochs

    def to_interactions_(self, x):
        (interactions, _) = data.build_interactions(x)
        return interactions
    
    def fit(self, x, y=None):
        interactions = self.to_interactions_(x)
        self.model.fit(interactions, epochs=self.epochs)
        
    def score(self, x):
        interactions = self.to_interactions_(x)
        return auc_score(self.model, interactions).mean()

        # k items with interaction / k items 
#         return precision_at_k(self.model, interactions, k=1).mean()


SyntaxError: non-default argument follows default argument (<ipython-input-119-d29f4790cff9>, line 4)

In [104]:
estimator = LightEstimator(data)

estimator.fit(train)

estimator.score(train), estimator.score(test)
# 1.4% odds, much better than random (0.4) but still perhaps too low to be useful

(0.8876644, 0.8609887)

In [80]:
pair_df = pd.read_csv('../pair_df.csv')
pair_df['comment_user_id'] = pair_df['user_id']
del pair_df['user_id']
pair_df['comment_focus_id'] = pair_df['subject_id']
del pair_df['subject_id']
pair_df.head()

,comment_user_id,comment_focus_id
0,142382,36254885
1,1807448,25809637
2,1361053,27361502
3,1392258,35622163
4,1856537,28481842


In [81]:
# pair_df['weights_pair'] = -1.  # by default
df['weights'] = 1.
# del pair_df['weights']

In [82]:
def did_talk(user, galaxy):
    return np.any((df['comment_user_id'] == str(user)) & (df['comment_focus_id'] == str(galaxy)))  # df passed by closure, careful

In [83]:
df['comment_focus_id'] = df['comment_focus_id'].astype(int).astype(str)
df['comment_user_id'] = df['comment_user_id'].astype(int).astype(str)
pair_df['comment_focus_id'] = pair_df['comment_focus_id'].astype(int).astype(str)
pair_df['comment_user_id'] = pair_df['comment_user_id'].astype(int).astype(str)

In [84]:
df.head()

,index,board_id,board_title,board_description,discussion_id,discussion_title,comment_id,comment_body,comment_focus_id,comment_focus_type,comment_user_id,comment_user_login,comment_created_at,weights_comments,weights_comment,weights
0,0,1267,Notes,General comment threads about individual subjects,528398,Subject 16215478,876591,The zoom scale of this image is incorrect. I a...,16215478,Subject,54,klmasters,2018-01-22 19:45:35.679000+00:00,1.0,1.0,1.0
1,2,1267,Notes,General comment threads about individual subjects,528398,Subject 16215478,894381,Agree with @klmasters on scale difference. Ans...,16215478,Subject,1428920,MerylPG,2018-02-06 15:13:36.441000+00:00,1.0,1.0,1.0
2,19,1267,Notes,General comment threads about individual subjects,539357,Subject 16215369,894501,lumpy,16215369,Subject,3870,trevor_allen_faller,2018-02-06 16:38:47.715000+00:00,1.0,1.0,1.0
3,33,1267,Notes,General comment threads about individual subjects,539298,Subject 16215566,894950,salut bravo pour le nouveau serveur il est tou...,16215566,Subject,1337311,discala_,2018-02-06 20:13:57.133000+00:00,1.0,1.0,1.0
4,36,1267,Notes,General comment threads about individual subjects,539368,Subject 16215384,894974,Hmm @StephenHawking,16215384,Subject,1373986,Ghost_Sheep_SWR,2018-02-06 20:24:45.930000+00:00,1.0,1.0,1.0


In [85]:
pair_df.head()

,comment_user_id,comment_focus_id
0,142382,36254885
1,1807448,25809637
2,1361053,27361502
3,1392258,35622163
4,1856537,28481842


In [86]:
# df['weights_temp'] = 1.

In [89]:
inter_df = pd.merge(pair_df, df[['comment_user_id', 'comment_focus_id', 'weights']], how='left', on=['comment_user_id', 'comment_focus_id'])

In [90]:
len(inter_df)

409981

In [91]:
inter_df.sample(20)

,comment_user_id,comment_focus_id,weights
358,1606700,32499144,NaN
100266,1919600,34240353,NaN
307589,114979,26445936,NaN
224593,31153,37902445,NaN
120350,300080,21664593,NaN
185185,31153,28491347,NaN
399875,1361053,28478689,NaN
82424,1702083,25810873,NaN
195207,1635018,34060146,NaN
405257,1246201,28490954,NaN


In [92]:
weights = inter_df['weights']
weights

0         NaN
1         NaN
2         NaN
3         1.0
4         NaN
         ... 
409976    NaN
409977    NaN
409978    NaN
409979    NaN
409980    NaN
Name: weights, Length: 409981, dtype: float64

In [93]:
~pd.isnull(weights)

0         False
1         False
2         False
3          True
4         False
          ...  
409976    False
409977    False
409978    False
409979    False
409980    False
Name: weights, Length: 409981, dtype: bool

In [94]:
weights = inter_df['weights']
weights = weights.where(~pd.isnull(weights), other=-1.,)  # replace weights values with -1 where they did not talk
weights

0        -1.0
1        -1.0
2        -1.0
3         1.0
4        -1.0
         ... 
409976   -1.0
409977   -1.0
409978   -1.0
409979   -1.0
409980   -1.0
Name: weights, Length: 409981, dtype: float64

In [95]:
inter_df['weights'] = weights

In [96]:
inter_df['weights'].mean()

-0.7115866345025745

In [97]:
data = Dataset()
data.fit(inter_df['comment_user_id'], inter_df['comment_focus_id'])
data.model_dimensions()

(361, 106141)

In [114]:
train, test, data_pairs = get_train_test(inter_df)

409981
222167


In [99]:
len(set([x[0] for x in train]))

357

In [100]:
len(set([x[0] for x in train]).intersection([x[0] for x in test]))

344

In [101]:
len(set([x[1] for x in train]).intersection([x[1] for x in test]))

23398

In [102]:
len(set([x[1] for x in train]))

29079

In [105]:
(inter_df['comment_focus_id'].value_counts() == 1).sum()

30888

In [106]:
len(inter_df['comment_focus_id'].unique())

106141

In [107]:
np.mean([x[2] for x in train]), np.mean([x[2] for x in test])

(-0.7022612570541205, -0.7060359184408336)

In [108]:
(train_interactions, _) = data.build_interactions(train)
print(repr(train_interactions))

(test_interactions, _) = data.build_interactions(test)
print(repr(test_interactions))

<361x106141 sparse matrix of type '<class 'numpy.int32'>'
	with 177733 stored elements in COOrdinate format>
<361x106141 sparse matrix of type '<class 'numpy.int32'>'
	with 44434 stored elements in COOrdinate format>


In [110]:
estimator = LightEstimator(data, loss='logistic')

estimator.fit(train)

estimator.score(train), estimator.score(test)

# rank 1 score still seems super poor - but AUC is now good??

(0.9139584, 0.85021776)

In [ ]:
assert False

In [115]:
param_distributions = {
#     'C': scipy.stats.expon(scale=100),
#     'gamma': scipy.stats.expon(scale=.1),
#     'kernel': ['rbf'],
#     'class_weight':['balanced', None]
    'no_components': [3, 10, 50]
}

In [116]:
cv = RandomizedSearchCV(estimator, param_distributions, n_iter=10)
cv.fit(data_pairs)

/home/walml/anaconda3/envs/agnfinder/lib/python3.7/site-packages/sklearn/model_selection/_search.py:281: UserWarning: The total space of parameters 3 is smaller than n_iter=10. Running 3 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=LightEstimator(data=<lightfm.data.Dataset object at 0x7fb5a57638d0>,
                                            loss='logistic', no_components=10),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions={'no_components': [3, 10, 50]},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [117]:
sorted(cv.cv_results_.items())

[('mean_fit_time', array([0.3299808 , 0.32830114, 0.32712307])),
 ('mean_score_time', array([6.13791189, 6.06816068, 6.08909593])),
 ('mean_test_score', array([0.85045449, 0.85054018, 0.8501372 ])),
 ('param_no_components',
  masked_array(data=[3, 10, 50],
               mask=[False, False, False],
         fill_value='?',
              dtype=object)),
 ('params',
  [{'no_components': 3}, {'no_components': 10}, {'no_components': 50}]),
 ('rank_test_score', array([2, 1, 3], dtype=int32)),
 ('split0_test_score', array([0.84998202, 0.85101593, 0.8509655 ])),
 ('split1_test_score', array([0.85099459, 0.85052007, 0.85073084])),
 ('split2_test_score', array([0.85029   , 0.85098076, 0.85039943])),
 ('split3_test_score', array([0.85071158, 0.85078484, 0.84974629])),
 ('split4_test_score', array([0.85029423, 0.84939933, 0.84884393])),
 ('std_fit_time', array([0.00444182, 0.00718766, 0.004927  ])),
 ('std_score_time', array([0.05097427, 0.0152363 , 0.04428639])),
 ('std_test_score', array([0.000

In [118]:
cv.best_estimator_, cv.best_score_

(LightEstimator(data=<lightfm.data.Dataset object at 0x7fb5a29b7150>,
                loss='logistic', no_components=10),
 0.8505401849746704)

In [ ]:
def score_model(train, test):

    
    train_precision = precision_at_k(model, train, k=10).mean()
    test_precision = precision_at_k(model, test, k=10).mean()

    train_auc = auc_score(model, train).mean()
    test_auc = auc_score(model, test).mean()

    print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
    print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))
    
    return test_precision

In [ ]:
ratings_normal_dense = np.loadtxt('ratings_normal.npy')

In [ ]:
train = ratings_normal_dense[:2000]
test = ratings_normal_dense[2000:]

In [ ]:
train = coo_matrix(train)
test = coo_matrix(test)

In [ ]:
from lightfm import LightFM

model = LightFM(no_components=30)

In [ ]:
model.fit(train, epochs=20)

In [ ]:
user_ids = list(range(test.shape[0]))
item_ids = list(range(test.shape[1]))

In [ ]:
predictions = model.predict([0], item_ids)


In [ ]:
test.shape, train.shape

In [ ]:
test_precision = precision_at_k(model, test, k=5).mean()

In [ ]:
predictions

In [ ]:
plt.hist(predictions, bins=30)